In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import molmodmt as m3t
from simtk import unit

/home/diego/Myopt/Miniconda/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/yank-0.24.0-py3.7-linux-x86_64.egg/yank/experiment.py:1168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """)


### Equilibration NPT

In [3]:
system_minimized = m3t.convert('system_minimized.pdb', 'openmm.Modeller')

In [4]:
import numpy as np
import simtk.openmm.app as app
import simtk.openmm as mm
import simtk.unit as unit
from openmmtools.integrators import LangevinIntegrator, GeodesicBAOABIntegrator
from tqdm import tqdm

In [5]:
temperature = 300 * unit.kelvin
pressure = 1.0 * unit.atmosphere

In [6]:
forcefield_generator = app.ForceField("amber99sbildn.xml","tip3p.xml")

In [7]:
topology = system_minimized.topology
positions = system_minimized.positions

In [8]:
system = forcefield_generator.createSystem(topology,
                                           contraints=app.HBonds,
                                           nonbondedMethod=app.PME,
                                           nonbondedCutoff=1.0*unit.nanometers,
                                           rigidWater=True,
                                           ewaldErrorTolerance=0.0005
                                          )

In [9]:
## Thermodynamic State
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = temperature
pressure = pressure

In [10]:
## Barostat
barostat_frequency = 25 # steps
barostat = mm.MonteCarloBarostat(pressure, temperature, barostat_frequency)
system.addForce(barostat)

5

In [11]:
## Integrator
friction   = 1.0/unit.picosecond
step_size  = 2.0*unit.femtoseconds
integrator = LangevinIntegrator(temperature, friction, step_size)
integrator.setConstraintTolerance(0.00001)

In [12]:
## Platform
platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}

In [13]:
## Simulation
simulation = app.Simulation(topology, system, integrator, platform, properties)

In [14]:
simulation.context.setPositions(positions)
simulation.context.setVelocitiesToTemperature(temperature)

In [15]:
## First equilibration with heavy atoms contrained in not solvent molecules

In [16]:
time_equilibration = 1.0 * unit.nanoseconds
time_iteration = 0.2 * unit.picoseconds
number_iterations = int(time_equilibration/time_iteration)
steps_iteration = int(time_iteration/step_size)
steps_equilibration = number_iterations*steps_iteration

In [17]:
print("steps_equilibration", steps_equilibration)
print("steps_iteration", steps_iteration)
print("number_iterations", number_iterations)

steps_equilibration 500000
steps_iteration 100
number_iterations 5000


In [18]:
niters = number_iterations
data = dict()
data['time'] = unit.Quantity(np.zeros([niters], np.float64), unit.picoseconds)
data['potential'] = unit.Quantity(np.zeros([niters], np.float64), unit.kilocalories_per_mole)
data['kinetic'] = unit.Quantity(np.zeros([niters], np.float64), unit.kilocalories_per_mole)
data['volume'] = unit.Quantity(np.zeros([niters], np.float64), unit.angstroms**3)
data['density'] = unit.Quantity(np.zeros([niters], np.float64), unit.gram / unit.centimeters**3)
data['kinetic_temperature'] = unit.Quantity(np.zeros([niters], np.float64), unit.kelvin)

In [19]:
net_mass, n_degrees_of_freedom = m3t.get(system, net_mass=True, n_degrees_of_freedom=True)

In [21]:
net_mass = net_mass.in_units_of(unit.gram/unit.mole)/unit.AVOGADRO_CONSTANT_NA

In [ ]:
for iteration in tqdm(range(number_iterations)):
    
    integrator.step(steps_iteration)
    
    state = simulation.context.getState(getEnergy=True)
    time = state.getTime()
    potential_energy = state.getPotentialEnergy()
    kinetic_energy = state.getKineticEnergy()
    volume = state.getPeriodicBoxVolume()
    density = (net_mass / volume).in_units_of(unit.gram / unit.centimeter**3)
    kinetic_temperature = (2.0 * kinetic_energy / kB / n_degrees_of_freedom).in_units_of(unit.kelvin) # (1/2) ndof * kB * T = KE
    
    data['time'][iteration]=time
    data['potential'] = potential_energy
    data['kinetic'] = kinetic_energy
    data['volume'] = volume
    data['density'] = density
    data['kinetic_temperature'] = kinetic_temperature
    

  1%|          | 61/5000 [00:43<1:00:23,  1.36it/s]

In [ ]:
simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, potentialEnergy=True, temperature=True))
simulation.step(1000)